In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import time

/Users/zhaoxiaoxiang/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/zhaoxiaoxiang/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/zhaoxiaoxiang/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/zhaoxiaoxiang/anaconda3/lib/python3.7/site-packages

In [3]:
#tensorflow to get handwriting datasets
mnist = input_data.read_data_sets('official/mnist/dataset.py', one_hot=True) 

Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting official/mnist/dataset.py/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting official/mnist/dataset.py/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting official/mnist/dataset.py/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting official/mnist/dataset.py/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [4]:
mnist.train.images.shape

(55000, 784)

In [5]:
mnist.train.labels.shape

(55000, 10)

In [6]:
batch_size = 128
X = tf.placeholder(tf.float32, [batch_size, 784], name='X_placeholder') 
Y = tf.placeholder(tf.int32, [batch_size, 10], name='Y_placeholder')

In [7]:
w = tf.Variable(tf.random_normal(shape=[784, 10], stddev=0.01), name='weights')
b = tf.Variable(tf.zeros([1, 10]), name="bias")

Instructions for updating:
Colocations handled automatically by placer.


In [8]:
logits = tf.matmul(X, w) + b 

In [9]:
entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y, name='loss')

loss = tf.reduce_mean(entropy)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [10]:
learning_rate = 0.01
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)

In [11]:
n_epochs = 30

with tf.Session() as sess:
	# 在Tensorboard里可以看到图的结构
	writer = tf.summary.FileWriter('./graphs/logistic_reg', sess.graph)

	start_time = time.time()
	sess.run(tf.global_variables_initializer())	
	n_batches = int(mnist.train.num_examples/batch_size)
	for i in range(n_epochs): # 迭代这么多轮
		total_loss = 0

		for _ in range(n_batches):
			X_batch, Y_batch = mnist.train.next_batch(batch_size)
			_, loss_batch = sess.run([optimizer, loss], feed_dict={X: X_batch, Y:Y_batch}) 
			total_loss += loss_batch
		print('Average loss epoch {0}: {1}'.format(i, total_loss/n_batches))

	print('Total time: {0} seconds'.format(time.time() - start_time))

	print('Optimization Finished!')

	# 测试模型
	
	preds = tf.nn.softmax(logits)
	correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(Y, 1))
	accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32))
	
	n_batches = int(mnist.test.num_examples/batch_size)
	total_correct_preds = 0
	
	for i in range(n_batches):
		X_batch, Y_batch = mnist.test.next_batch(batch_size)
		accuracy_batch = sess.run([accuracy], feed_dict={X: X_batch, Y:Y_batch}) 
		total_correct_preds += accuracy_batch[0]
	
	print('Accuracy {0}'.format(total_correct_preds/mnist.test.num_examples))

	writer.close()

Average loss epoch 0: 0.3678491050268942
Average loss epoch 1: 0.29443862443759444
Average loss epoch 2: 0.2850705863851489
Average loss epoch 3: 0.27839196419521367
Average loss epoch 4: 0.2761933158883404
Average loss epoch 5: 0.27381095569797886
Average loss epoch 6: 0.2712877638342775
Average loss epoch 7: 0.26652871796509603
Average loss epoch 8: 0.2685324200591841
Average loss epoch 9: 0.2659802682228855
Average loss epoch 10: 0.2612340056048684
Average loss epoch 11: 0.26486661262584454
Average loss epoch 12: 0.262163507354843
Average loss epoch 13: 0.26054460994708234
Average loss epoch 14: 0.2578110230309424
Average loss epoch 15: 0.26146707989466494
Average loss epoch 16: 0.25797689291876513
Average loss epoch 17: 0.258679165958594
Average loss epoch 18: 0.25603301156234076
Average loss epoch 19: 0.25821498649451
Average loss epoch 20: 0.25608044892271636
Average loss epoch 21: 0.2558372005959213
Average loss epoch 22: 0.2574540524111761
Average loss epoch 23: 0.2535373772948